In [2]:
from sklearn.base import BaseEstimator
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
import pandas as pd
import collections as cl
import numpy as np


class HPClassifier(BaseEstimator):
    def __init__(self, n_Samples=None):
        super().__init__()
        self.n_Samples = n_Samples
        self.classifiers = []
        self.frequencies = []

    def fit(self, data, target):
        classifiers = []

        self.frequencies = cl.Counter(target)

        for nr_Index in range(self.n_Samples):
            if nr_Index != 0:
                data, target = resample(data, target, random_state=nr_Index-1)

            gaussianClassifier = GaussianNB()
            decisionClassifier = DecisionTreeClassifier()
            knnClassifier = KNeighborsClassifier(n_neighbors=1)

            gaussianClassifier.fit(data, target)
            decisionClassifier.fit(data, target)
            knnClassifier.fit(data, target)

            classifiers.append(gaussianClassifier)
            classifiers.append(decisionClassifier)
            classifiers.append(knnClassifier)

        self.classifiers = classifiers

    def predict(self, data):
        result = []
        predictions = []

        for classifier in self.classifiers:
            prediction = classifier.predict(data)
            predictions.append(prediction)

        dataFrame = pd.DataFrame(data=predictions)
        data = np.asarray(data)

        for index in dataFrame.columns:
            column = dataFrame[index]
            choice = cl.Counter(column).most_common(1)[0][0]
            self.getBetterChoice(column)
            result.append(choice)

        return np.asarray(result)

    def getBetterChoice(self, column):
        frequencies = cl.Counter(column).items()
        betterFrequency = 0
        better = None
        print(self.frequencies)
        for key, value in frequencies:
            if(value > betterFrequency):
                better = key
                betterFrequency = value
            elif(value == betterFrequency):
                for selfKey, selfValue in self.frequencies.most_common():
                    if(selfKey == better):
                        break
                    if(selfKey == key):
                        better = key
                        betterFrequency = value

        return better

In [7]:
from sklearn import datasets
from scipy import stats
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import seaborn as sns
import numpy as np


def Get_Stats(data, target, classifier, grade={'classifier__n_estimators': [10, 25, 50, 100]}):
    scaler = StandardScaler()

    pipeline = Pipeline([("scaler", scaler), ('classifier', classifier)])

    grade = grade

    gridSearch = GridSearchCV(pipeline, param_grid=grade,
                              scoring='accuracy', cv=4)

    cv = RepeatedStratifiedKFold(
        n_splits=10, n_repeats=3, random_state=36851234)

    scores = cross_val_score(
        gridSearch, data, target, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

    mean = scores.mean()
    std = scores.std()
    lower, upper = stats.norm.interval(
        0.95, loc=mean, scale=std / np.sqrt(len(scores)))

    return [scores, mean, std, lower, upper]


def Get_Digits():
    digits = datasets.load_digits()
    target = digits.target
    data = digits.data
    return data, target


def Get_Wine():
    digits = datasets.load_wine()
    target = digits.target
    data = digits.data
    return data, target


def Get_Breast_Cancer():
    digits = datasets.load_breast_cancer()
    target = digits.target
    data = digits.data
    return data, target


def Get_Boxplot(results, names=["Bagging", "AdaBoost", "Random Forest", "HP"]):
    for result in results:
        print("Results:", result)
        result = [x for xs in result for x in xs]

    results = {'scores': results, 'names': names}
    data = pd.DataFrame(results, columns=["scores", "names"])
    sns.boxplot(x='names', y='scores', data=data)


def Calculate_Digits_Classifiers():
    data, target = Get_Digits()
    classifier = BaggingClassifier()
    baggingResults = Get_Stats(data, target, classifier)

    data, target = Get_Digits()
    classifier = AdaBoostClassifier()
    adaBoostResults = Get_Stats(data, target, classifier)

    data, target = Get_Digits()
    classifier = RandomForestClassifier()
    randomForestResults = Get_Stats(data, target, classifier)

    data, target = Get_Digits()
    classifier = HPClassifier()
    hpResults = Get_Stats(data, target, classifier, {
                          'classifier__n_Samples': [1, 3, 5, 7]})

    print(baggingResults)
    print(adaBoostResults)
    print(randomForestResults)
    print(hpResults)

    return [baggingResults, adaBoostResults, randomForestResults, hpResults]


def Calculate_Wine_Classifiers():
    data, target = Get_Wine()
    classifier = BaggingClassifier()
    baggingResults = Get_Stats(data, target, classifier)

    data, target = Get_Wine()
    classifier = AdaBoostClassifier()
    adaBoostResults = Get_Stats(data, target, classifier)

    data, target = Get_Wine()
    classifier = RandomForestClassifier()
    randomForestResults = Get_Stats(data, target, classifier)

    data, target = Get_Wine()
    classifier = HPClassifier()
    hpResults = Get_Stats(data, target, classifier, {
                          'classifier__n_Samples': [1, 3, 5, 7]})

    print(baggingResults)
    print(adaBoostResults)
    print(randomForestResults)
    print(hpResults)

    return [baggingResults, adaBoostResults, randomForestResults, hpResults]


def Calculate_Breast_Cancer_Classifiers():
    data, target = Get_Breast_Cancer()
    classifier = BaggingClassifier()
    baggingResults = Get_Stats(data, target, classifier)

    data, target = Get_Breast_Cancer()
    classifier = AdaBoostClassifier()
    adaBoostResults = Get_Stats(data, target, classifier)

    data, target = Get_Breast_Cancer()
    classifier = RandomForestClassifier()
    randomForestResults = Get_Stats(data, target, classifier)

    data, target = Get_Breast_Cancer()
    classifier = HPClassifier()
    hpResults = Get_Stats(data, target, classifier, {
                          'classifier__n_Samples': [1, 3, 5, 7]})

    print(baggingResults)
    print(adaBoostResults)
    print(randomForestResults)
    print(hpResults)

    return [baggingResults, adaBoostResults, randomForestResults, hpResults]


def Get_Accuracies(results):
    accuracies = []

    for item in results:
        accuracies.append(item[0])

    return accuracies


# Get_Boxplot(Get_Accuracies(Calculate_Digits_Classifiers()))
Get_Boxplot(Get_Accuracies(Calculate_Wine_Classifiers()))
# Get_Boxplot(Get_Accuracies(Calculate_Breast_Cancer_Classifiers()))


[array([1.        , 1.        , 0.94444444, 1.        , 0.83333333,
       1.        , 1.        , 1.        , 0.94117647, 0.88235294,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.77777778, 1.        , 0.83333333, 1.        , 0.94117647,
       0.94444444, 1.        , 0.94444444, 0.94444444, 1.        ,
       0.94444444, 1.        , 0.94444444, 1.        , 1.        ]), 0.9625272331154683, 0.058335748919144434, 0.9416524370483708, 0.9834020291825658]
[array([0.88888889, 0.88888889, 0.94444444, 1.        , 0.88888889,
       0.94444444, 0.83333333, 1.        , 0.94117647, 0.94117647,
       0.72222222, 0.83333333, 0.88888889, 0.94444444, 1.        ,
       0.88888889, 0.88888889, 0.88888889, 0.94117647, 1.        ,
       1.        , 1.        , 0.83333333, 1.        , 0.94444444,
       0.83333333, 0.77777778, 0.94444444, 0.82352941, 1.        ]), 0.9141612200435728, 0.07302505959298042, 0.8880300182499015, 0.9402924218372442]
[array([1.        , 1.     

TypeError: 'numpy.float64' object is not iterable